In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import datetime

# Start spark session
startTime = datetime.datetime.now()

spark = SparkSession \
    .builder\
    .appName("Anime importer") \
    .master("spark://spark-master:7077") \
    .config("spark.driver.memory", "15g") \
    .config("spark.jars", "/extra_jars/neo4j-connector-apache-spark_2.12-4.0.1_for_spark_3.jar") \
    .config("neo4j.url", "bolt://neo4j:7687")\
    .config("neo4j.authentication.type", "basic")\
    .config("neo4j.authentication.basic.username", "neo4j")\
    .config("neo4j.authentication.basic.password", "password")\
    .getOrCreate()

print(datetime.datetime.now()-startTime)

0:00:03.735518


In [2]:
# Read animes with cypher test
startTime = datetime.datetime.now()
print(startTime)

(spark.read.format("org.neo4j.spark.DataSource")
#  .option("labels", ":Anime") 
 .option("query", "MATCH (a:Anime) WITH a LIMIT 2 RETURN id(a) as id, a.title as title")
 .load()
 .show())

print(datetime.datetime.now()-startTime)

2021-06-17 18:18:45.538230
+------+--------------------+
|    id|               title|
+------+--------------------+
|108712|Tezuka Osamu no B...|
|108713|Kono Danshi, Maho...|
+------+--------------------+

0:00:05.379765


In [20]:
# Read animes with cypher 
startTime = datetime.datetime.now()

query = """
    CALL gds.graph.create('got-interactions', 'Person', {
      INTERACTS: {
        orientation: 'UNDIRECTED'
      }
    })
    YIELD graphName, nodeCount, relationshipCount, createMillis
    RETURN graphName, nodeCount, relationshipCount, createMillis
"""

query = """
    MATCH (n) WITH n LIMIT 10 
    RETURN n
"""

query = """CALL gds.graph.create.cypher(
        '20001r_graph', 
        'MATCH (n) RETURN id(n) AS id', 
        'MATCH (n:User)-[r:WATCHED]->(m:Anime) 
            RETURN id(n) AS source, 
            id(m) AS target, 
            r.my_watched_episodes AS weight 
        LIMIT 20001')
        """

query = "CALL gds.graph.create.cypher('20004r_graph', 'MATCH (n) RETURN id(n) AS id', 'MATCH (n:User)-[r:WATCHED]->(m:Anime) RETURN id(n) AS source, id(m) AS target, r.my_watched_episodes AS weight LIMIT 20002') YIELD graphName RETURN graphName"
#         ,  configuration: {validateRelationships:false}

CALL gds.graph.create.cypher('full_graph_retry', 'MATCH (n) RETURN id(n) AS id', 'MATCH (n:User)-[r:WATCHED]->(m:Anime) RETURN id(n) AS source, id(m) AS target, r.my_watched_episodes AS weight')

 # schema still not working!!
temp = (spark.read.format("org.neo4j.spark.DataSource")
 .option("query", "RETURN 'ok'")
 .load())
temp.printSchema()

(spark.read.format("org.neo4j.spark.DataSource")
  # https://neo4j.com/developer/spark/faq/#graph-already-exists
 # https://neo4j.com/developer/spark/gds/
 # schema still not working!!
 .schema(StructType([StructField("graphName",StringType())]))
#  .schema(temp.schema)
 .option("query", query)
 .option("partitions", "1") 
 .load()
 .show())

print(datetime.datetime.now()-startTime)

root
 |-- 'ok': string (nullable = true)



Py4JJavaError: An error occurred while calling o324.load.
: java.lang.UnsupportedOperationException: DataSource source does not support user-specified schema.
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2Utils$.getTableFromProvider(DataSourceV2Utils.scala:76)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$1(DataFrameReader.scala:256)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:230)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:203)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gatew